<a href="https://colab.research.google.com/github/MJMortensonWarwick/AnalyticsInPractice2425/blob/main/4_1_Linear_Models_(ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Models in Machine Learning Regression

In this notebook we will work through a full ML approach for a regression problem, using _linear regression_, _LASSO regression (L1)_, _Ridge Regression (L2)_ and _ElasticNet_.

We'll begin with getting a dataset together:

In [1]:
import pandas as pd
import numpy as np

# read in the data
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv", header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


To keep focused on the task in hand we are treating this purely as a numerical problem with no domain injection ... i.e. we intentionally exclude (most) feature engineering or even feature names. Our goal is to predict item #13 from the remaining 13 features ... i.e.:

$item_{13} = α + \beta_1item_0 + ... + \beta_{13}item_{12} + ϵ$

We need to separate out our target variable (item #13):

In [2]:
# serparate the x and Y values
# \n means line break
x_values = df.drop([13], axis = 1)
print(f'X values: \n {x_values.head()}\n')

y_value = df[13]
print(f'Y value: \n {y_value[0:5]}')

X values: 
         0     1     2   3      4      5     6       7   8      9     10  \
0  0.00632  18.0  2.31   0  0.538  6.575  65.2  4.0900   1  296.0  15.3   
1  0.02731   0.0  7.07   0  0.469  6.421  78.9  4.9671   2  242.0  17.8   
2  0.02729   0.0  7.07   0  0.469  7.185  61.1  4.9671   2  242.0  17.8   
3  0.03237   0.0  2.18   0  0.458  6.998  45.8  6.0622   3  222.0  18.7   
4  0.06905   0.0  2.18   0  0.458  7.147  54.2  6.0622   3  222.0  18.7   

       11    12  
0  396.90  4.98  
1  396.90  9.14  
2  392.83  4.03  
3  394.63  2.94  
4  396.90  5.33  

Y value: 
 0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: 13, dtype: float64


_Note: I would recommend we at least normalise the data and do our basic EDA checks. However, for speed we will ignore this._

## Train-Test split
As per the slides, our next step will be to split the data such that part can be used for training the model, and part can be reserved for testing the model. We can do this with just a couple of lines of code:

In [3]:
# split data into training and test
from sklearn.model_selection  import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_values, y_value, test_size = 0.2, random_state=1234)

# print the shapes to check everything is OK
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(404, 13)
(102, 13)
(404,)
(102,)


In our code we have split using an 80:20 ratio (80% of the data for training; 20% for testing). We specify _random\_state_ as a fixed number so that this split will be the same each time (the splitting algorithm is based on random numbers). We can confirm this has worked by looking at the size of our different datasets:


*   _X\_train_ (the $x$ values we use for training) is 404 rows and 13 columns. 13x columns is what we would expect;
*   _X\_test_ (the $x$ values we use for testing) is 102 rows and 13 columns. Again we expect the 13x columns, but also we can compare the number of rows with _X\_train_ ... 102 rows is approximately 20% of the total;
*   _Y\_train_ (the $Y$ values we use for training) is 404 rows and a single column - again, what we would expect;
*   _Y\_test_ (the $Y$ values we use for testing) is 102 rows and a single columns. All seems to be correct!

## Linear Regression
We'll begin with a standard regression model (i.e. no regularisation ... just pure Ordinary Least Squares). We first need to specify the model:

In [4]:
from sklearn.linear_model import LinearRegression as LR

# create the model
lr_algo = LR()
lr_algo

LinearRegression()

We have now created an object (_lr\_algo_) which is an instance of Scikit Learn's _LinearRegression_ class. This is the standard linear regression algorithm which we can train as a model using our data. Printed to screen we get an orange box. If you hover over the information icon ("i") you'll see it is "Not fitted" - meaning not trained on any data. Let's fix that:

In [5]:
lr_model = lr_algo.fit(X_train, Y_train)
lr_model

LinearRegression()

We have now successfully trained a model! Now the box is blue, and hovering over the infomation icon shows "fitted" (i.e. trained). So what is our model?

In [6]:
# get and print the alpha value
alpha = lr_model.intercept_
print(f'Alpha: {alpha}')

# we'll run a count variable we can update in our for loop
# this will allow us to print the beta value name each time
count = 1

# we expect 13x beta values so we'll loop through and print
betas = lr_model.coef_
for beta in betas:
  # use the round function to round the coefficient to 2 decimal places
  print(f'Beta #{count}: {round(beta, 2)}')
  # increase the count value by one for the next beta name
  count += 1

Alpha: 45.73718122895507
Beta #1: -0.1
Beta #2: 0.06
Beta #3: 0.03
Beta #4: 3.0
Beta #5: -20.41
Beta #6: 2.89
Beta #7: -0.01
Beta #8: -1.76
Beta #9: 0.34
Beta #10: -0.01
Beta #11: -1.02
Beta #12: 0.01
Beta #13: -0.53


I.e. we have a regression model of:

$Y = 45.74 - 0.1x_1 + 0.06x_2 \text{ } \text{ } ... \text{ } - 0.53x_{13}$

But how does it perform? Let's check the metrics:

In [7]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2

# predict the test data
predict = lr_model.predict(X_test)

# seperate the first five predictions and the first five real values in Y_test
for i in range(5):
  print(f'Predicted: {round(predict[i],2)}')
  print(f'Real: {round(Y_test.iloc[i],2)}')
  print("\n")

print("\n")

# calculate each metric by comparing the prediction with the real value in Y_test
print(f'MSE: {round(MSE(Y_test, predict),2)}')
print(f'RMSE: {round(RMSE(Y_test, predict),2)}')
print(f'MAE: {round(MAE(Y_test, predict),2)}')
print(f'R2: {round(R2(Y_test, predict),2)}')

Predicted: 21.32
Real: 33.0


Predicted: 23.91
Real: 27.5


Predicted: 12.27
Real: 5.6


Predicted: 22.08
Real: 21.2


Predicted: 14.21
Real: 14.9




MSE: 23.96
RMSE: 4.9
MAE: 3.58
R2: 0.77


Our first function ("predict = ...") uses our model to predict X_test (our test data). We print the first five to screen for visual inspection along with the first five values in Y_test (the real values). Most look OK!

However, there are more than 100 in our test set, better to use metrics to estiamte performance. We display values for MSE, RMSE, MAE and the $R^2$. Overall these look successful - generally a $R^2$ above 0.7 is considered very good.

But how will these perform with regularisation?

### FIRST TASK
Your first task is to "fit" (train) models for our three remaining algorithms (_LASSO regression_, _ridge regression_ and _ElasticNet_). You can use the following code to start you off:

In [8]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet

# LASSO
# set alpha at 0.5 ... half facilitator (OLS) and half policeman (L1)
l1_algo = Lasso(alpha=.5)

# Ridge
# set alpha at 0.5 ... half facilitator (OLS) and half policeman (L2)
l2_algo = Ridge(alpha=.5)

# ElasticNet
# set alpha at 0.5 ... half facilitator (OLS) and half policeman (L1 + L2)
# set the proportion of L1 vs L2 as 50/50
elastic_algo = ElasticNet(alpha=.5, l1_ratio=.5)

In [9]:
# Your code here

### SECOND TASK
Compare the beta values (co-efficients) for each of the models. Are the results as expected?

In [10]:
# Your code here

### THIRD TASK
Compare the metrics (MSE, RMSE, MAE and $R^2$) for each of the models. Which one is best?

In [11]:
# Your code here

### FOURTH TASK (optional / if you have time)
What impact does it have to change the hyperparameters of the regularised models (i.e. $\alpha$ for all three and the $L1\_ratio$ for _ElasticNet_). Generally I would keep $\alpha$ between 0.1 and 20; and the $L1\_ratio$ has to be between 0-1.

In [12]:
# Your code here